In [ ]:
!pip install -U xformers --index-url https://download.pytorch.org/whl/cu121
!pip install --no-deps packaging ninja einops flash-attn trl peft accelerate bitsandbytes
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [ ]:
import torch
import os
from google.colab import userdata
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig
from trl import SFTTrainer

In [ ]:
import os
os.environ["HF_TOKEN"]=userdata.get("HF_TOKEN")

In [ ]:
model_id="mistralai/Mistral-7B-Instruct-v0.3"
bnb_config=BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
tokenizer=AutoTokenizer.from_pretrained(model_id,use_auth_token=True)

In [ ]:
model=AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map={"": 0},
    token=os.environ["HF_TOKEN"]
)

In [ ]:
text="who is the prime minster of india"
device="cuda:0"
input=tokenizer(text,return_tensors="pt").to(device)
output=model.generate(**input,max_new_tokens=100)
answer=tokenizer.decode(output[0],skip_special_tokens=True)

In [ ]:
answer

In [ ]:
os.environ["WANDB_DISABLED"]="false"
lora_config=LoraConfig(
    r=8,
    target_modules=["q_proj","o_proj","k_proj","v_proj","gate_proj","up_proj","down_proj"],
    task_type="CAUSAL_LM"
)

In [ ]:
from datasets import Dataset
dataset1 = [
    {
        "text": "<s>[INST] Can you write a short introduction about Manu, the famous Generative AI developer from Vaikom? [/INST] "
                "Manu is a renowned Generative AI developer from Vaikom, known for his contributions to the field of artificial intelligence. "
                "He specializes in building AI models, developing innovative applications, and exploring cutting-edge technologies in the domain of generative AI. "
                "Manu is currently pursuing his studies at TOCH Institute of Science and Technology, where he continues to expand his expertise and work on impactful AI-driven projects. "
                "His dedication to the field has made him a well-recognized name in the AI community. </s> "
                "<s>[INST] Now explain it to a dog [/INST] "
                "Manu is really smart! He teaches computers how to create things, like pictures and words, just like how you learn tricks. "
                "He works very hard at a big school to get even better. If he were a dog trainer, he would teach computers to fetch data instead of sticks! </s>"
    },
    {
        "text": "<s>[INST] What is artificial intelligence? [/INST] "
                "Artificial intelligence (AI) is the simulation of human intelligence in machines. It enables computers to perform tasks such as learning, reasoning, problem-solving, and understanding language. "
                "AI is used in various applications, from chatbots and self-driving cars to medical diagnosis and creative tools. </s> "
                "<s>[INST] Explain it to a child [/INST] "
                "AI is like a robot brain that can learn things and help people. Imagine a smart toy that talks to you and remembers your favorite games—that's AI! </s>"
    }
]

hf_dataset = Dataset.from_list(dataset1)
def format_func(example):
    return example['text']


In [ ]:
import transformers
trainer = SFTTrainer(
    model=model,
    train_dataset=hf_dataset,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=2,
        max_steps=100,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit"
    ),
    peft_config=lora_config,
    formatting_func=format_func
)

In [ ]:
trainer.train()

In [ ]:
text="Can you write a short introduction about Manu"
device="cuda:0"
input=tokenizer(text,return_tensors="pt").to(device)
output=model.generate(**input,max_new_tokens=100)
answer=tokenizer.decode(output[0],skip_special_tokens=True)

In [ ]:
answer